In [1]:
import os

In [2]:
%pwd

'c:\\Users\\asdaw\\Desktop\\Projects\\KidneyScan\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\asdaw\\Desktop\\Projects\\KidneyScan'

In [11]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    history_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: list
    params_is_augmentation: bool
    

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        params_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(
            self.config.data_ingestion.unzip_dir,
             "kidney-ct-scan-image"
        )
        
        create_directories([
            Path(training.root_dir)
        ])
        
        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            history_path = Path(training.history_path),
            updated_base_model_path = Path(params_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_image_size = params.IMAGE_SIZE,
            params_is_augmentation = params.AUGMENTATION
        )
        
        return training_config

In [21]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import math
import json

In [25]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    @staticmethod
    def save_history(path: Path, history: dict):
        path.parent.mkdir(parents=True, exist_ok=True)
    
        # Convert numpy types to native Python types
        cleaned_history = {
            key: [float(value) for value in values]
            for key, values in history.items()
        }
    
        with open(path, "w") as f:
            json.dump(cleaned_history, f, indent=4)

    
    def get_callbacks(self):
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=5,
            restore_best_weights=True
        )

        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.2,
            patience=3,
            min_lr=1e-6
        )
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.trained_model_path,
            monitor="val_loss",
            save_best_only=True
        )
        
        return [early_stopping, reduce_lr, checkpoint]

    
    def train(self):
        self.steps_per_epoch = math.ceil(
        self.train_generator.samples / self.train_generator.batch_size
        )
        self.validation_steps = math.ceil(
            self.valid_generator.samples / self.valid_generator.batch_size
        )
        callbacks = self.get_callbacks()

        history = self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,  # upper bound only
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            callbacks=callbacks
        )    

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

        self.save_history(
            path=self.config.history_path,
            history=history.history
        )

In [26]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
     
except Exception as e:
    raise e


[2025-12-26 18:45:24,384: INFO: common]: yaml file: C:\Users\asdaw\Desktop\Projects\KidneyScan\config\config.yaml loaded successfully]
[2025-12-26 18:45:24,386: INFO: common]: yaml file: C:\Users\asdaw\Desktop\Projects\KidneyScan\params.yaml loaded successfully]
[2025-12-26 18:45:24,388: INFO: common]: created directory at: artificats]
[2025-12-26 18:45:24,389: INFO: common]: created directory at: artifacts\training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/30
24/24 [==============================] - 35s 1s/step - loss: 9.3010 - accuracy: 0.6075 - val_loss: 1.4429 - val_accuracy: 0.8817 - lr: 0.0100
Epoch 2/30
24/24 [==============================] - 40s 2s/step - loss: 5.9662 - accuracy: 0.6882 - val_loss: 9.2679 - val_accuracy: 0.5054 - lr: 0.0100
Epoch 3/30
24/24 [==============================] - 40s 2s/step - loss: 4.5553 - accuracy: 0.7473 - val_loss: 2.6089 - val_accuracy: 0.5269 - lr: 0.0100
Epoch 4/30
24/24 [=====================